In [1]:
from __future__ import print_function, division

import tensorflow as tf

print(tf.__version__)

import keras

import matplotlib.pyplot as plt

import sys

from sklearn import preprocessing
from sklearn.cross_validation import train_test_split

import time
from tqdm import tqdm

from imutils import paths
from numpy import *
import numpy as np

from matplotlib.colors import Normalize

import argparse
import imutils,sklearn
import os, cv2, re, random, shutil, imageio, pickle

%matplotlib inline  

1.3.0


/home/huiqy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/huiqy/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
class CNN(object):
    def __init__(self, sess, epoch, batch_size, dataset_name, checkpoint_dir, log_dir, trainhist_dir):
        self.sess = sess
        self.dataset_name = dataset_name
        self.checkpoint_dir = checkpoint_dir
        self.log_dir = log_dir
        self.trainhist_dir = trainhist_dir
        self.epoch = epoch
        self.batch_size = batch_size

        # parameters
        self.input_height = 128
        self.input_width = 128
        self.c_dim = 3  # color dimension
        self.nb_class = 4
        
        # number of convolutional filters to use  
        self.nb_CNN = [32, 64, 64, 64, 128]  
        # number of dense filters to use  
        self.nb_Dense = [256] 
        # size of pooling area for max pooling  
        self.pool_size = (2, 2)  
        # convolution kernel size  
        self.kernel_size = (3, 3)
        self.batch_normalization_control = True
        
        # name for checkpoint
        self.model_name = 'CNN_Canny&Pyramid_L_C%d_D%d_Kernel(%d,%d)_%d_lrdecay' % (len(self.nb_CNN), len(self.nb_Dense),
                                                          self.kernel_size[0], self.kernel_size[1], max(self.nb_CNN))

        # train
        #设置一个全局的计数器
        self.global_step = tf.Variable(0, trainable=False)
        self.lr = tf.train.exponential_decay(0.001, 
                                             global_step=self.global_step, 
                                             decay_steps=10, 
                                             decay_rate=0.9, 
                                             staircase=True)
        self.beta1 = 0.5
        #max model to keep saving
        self.max_to_keep = 300
        
        # test

        #load_flower_data
        self.train_x = np.load('../trainImage.npy')
        self.train_y = np.load('../trainLabels.npy')
        self.test_x = np.load('../testImage.npy')
        self.test_y = np.load('../testLabels.npy')
        self.train_x_f1 = np.load('../trainImage_canny.npy')
        self.test_x_f1 = np.load('../testImage_canny.npy')
        self.train_x_f2 = np.load('../trainImage_pyramid_L.npy')
        self.test_x_f2 = np.load('../testImage_pyramid_L.npy')
        self.train_x_f3 = np.load('../trainImage_pyramid_G.npy')
        self.test_x_f3 = np.load('../testImage_pyramid_G.npy')

        
        #记录
        self.train_hist = {}
        self.train_hist['losses'] = []
        self.train_hist['accuracy'] = []
        self.train_hist['learning_rate'] = []
        self.train_hist['per_epoch_ptimes'] = []
        self.train_hist['total_ptime'] = []
        
        # get number of batches for a single epoch
        self.num_batches_train = len(self.train_x) // self.batch_size
        self.num_batches_test= len(self.test_x) // self.batch_size

    def cnn_model(self, x, x_f1, x_f2, x_f3, keep_prob, is_training=True, reuse=False):
        with tf.variable_scope("cnn", reuse=reuse):
             
            #初始化参数
            W = tf.truncated_normal_initializer(mean=0.0, stddev=0.02)
            B = tf.constant_initializer(0.0)
        
            print("CNN:x",x.get_shape()) # 128, 128, 3 
            print("CNN:x_f1",x_f1.get_shape()) 
            print("CNN:x_f2",x_f2.get_shape())  
            print("CNN:x_f3",x_f3.get_shape())  
            
            #输入x,卷积核为3*3 输出维度为32
            net1_1 = tf.layers.conv2d(inputs = x,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_1'               # 命名用于获取变量
                                    )
            print("CNN:",net1_1.get_shape())
            
            #输入x_f1,卷积核为3*3 输出维度为32
            net1_2 = tf.layers.conv2d(inputs = x_f1,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_2'               # 命名用于获取变量
                                    )
            print("CNN:",net1_2.get_shape())
            
            #输入x_f2,卷积核为3*3 输出维度为32
            net1_3 = tf.layers.conv2d(inputs = x_f2,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_3'               # 命名用于获取变量
                                    )
            print("CNN:",net1_3.get_shape())
            
            #输入x_f3,卷积核为3*3 输出维度为32
            net1_4 = tf.layers.conv2d(inputs = x_f3,                 # 输入,
                                    filters = self.nb_CNN[0],      # 卷积核个数,
                                    kernel_size = self.kernel_size,          # 卷积核尺寸
                                    strides = (1, 1),
                                    padding = 'same',              # padding方法
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer = None,
                                    bias_regularizer = None,
                                    activity_regularizer = None,
                                    name = 'conv_1_4'               # 命名用于获取变量
                                    )
            print("CNN:",net1_4.get_shape())

            #把数据和边缘进行连接
            net = tf.concat([net1_1, net1_2], 3)
            net = tf.concat([net, net1_3], 3)
            net = tf.concat([net, net1_4], 3)
            net = tf.layers.batch_normalization(net, training=is_training)
            net = tf.nn.relu(net, name = 'relu_conv_1')
            print("CNN:",net.get_shape())
            net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_1'
                                             )
            
            for i in range(2,len(self.nb_CNN)+1):
                net = tf.layers.conv2d(inputs = net,                 # 输入,
                                       filters = self.nb_CNN[i-1],      # 卷积核个数,
                                       kernel_size = self.kernel_size,          # 卷积核尺寸
                                       strides = (1, 1),
                                       padding = 'same',              # padding方法
                                       kernel_initializer = W,
                                       bias_initializer = B,
                                       kernel_regularizer = None,
                                       bias_regularizer = None,
                                       activity_regularizer = None,
                                       name = 'conv_'+ str(i)        # 命名用于获取变量
                                       )
                print("CNN:",net.get_shape())
                if self.batch_normalization_control:
                    net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_conv_' + str(i))
                net = tf.layers.max_pooling2d(inputs = net,
                                              pool_size = self.pool_size,
                                              strides = (2, 2),
                                              padding = 'same',
                                              name = 'pool_conv_' + str(i)
                                             )
                print("CNN:",net.get_shape())
            
            #flatten
            net = tf.reshape(net, [-1, int(net.get_shape()[1]*net.get_shape()[2]*net.get_shape()[3])],name='flatten')
            print("CNN:",net.get_shape())
            
            #dense layer
            for i in range(1,len(self.nb_Dense)+1):
                net = tf.layers.dense(inputs = net,
                                    units = self.nb_Dense[i-1],
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_' + str(i)
                                    )
#                 net = tf.layers.batch_normalization(net, training=is_training)
                net = tf.nn.relu( net, name = 'relu_dense_' + str(i))
                net = tf.layers.dropout(inputs = net,
                                        rate=keep_prob,
                                        noise_shape=None,
                                        seed=None,
                                        training = is_training,
                                        name= 'dropout_dense_' + str(i)
                                        )
            #output
            logit = tf.layers.dense(inputs = net,
                                    units = self.nb_class,
                                    kernel_initializer = W,
                                    bias_initializer = B,
                                    kernel_regularizer=None,
                                    bias_regularizer=None,
                                    activity_regularizer=None,
                                    name = 'dense_output'
                                    )
            out_logit = tf.nn.softmax(logit, name="softmax")
            print("CNN:out_logit",out_logit.get_shape())
            print("------------------------")    

            return out_logit, logit


    def build_model(self):

        """ Graph Input """
        # images
        self.x = tf.placeholder(tf.float32, shape=[self.batch_size,self.input_height, self.input_width, self.c_dim], 
                                name='x_image')
        
        self.x_f1 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f1.shape[1], 
                                                     self.train_x_f1.shape[2], self.train_x_f1.shape[3]],
                                  name='x_f1')
        self.x_f2 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f2.shape[1], 
                                                     self.train_x_f2.shape[2], self.train_x_f2.shape[3]],
                                  name='x_f2')
        self.x_f3 = tf.placeholder(tf.float32, shape=[self.batch_size,self.train_x_f3.shape[1], 
                                             self.train_x_f3.shape[2], self.train_x_f3.shape[3]],
                          name='x_f3')

        self.y = tf.placeholder(tf.float32, shape=[self.batch_size, self.nb_class], name='y_label')
        
        self.keep_prob = tf.placeholder(tf.float32)
        
        self.add_global = self.global_step.assign_add(1)

        """ Loss Function """

        # output of cnn_model
        self.out_logit, self.logit = self.cnn_model(self.x, self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=True, reuse=False)
        
        self.loss_cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=self.y,
                                                                                         logits =self.logit))
        
        """ Training """
        # trainable variables into a group
        tf_vars = tf.trainable_variables()
        cnn_vars = [var for var in tf_vars if var.name.startswith('cnn')]

        # optimizers
        with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
            self.cnn_optim = tf.train.AdamOptimizer(self.lr, beta1=self.beta1).minimize(self.loss_cross_entropy,
                                                                                        var_list=cnn_vars)
        """" Testing """
        # for test
        # output of cnn_model
        self.out_logit_test, self.logit_test = self.cnn_model(self.x,  self.x_f1, self.x_f2, self.x_f3, self.keep_prob, is_training=False, reuse=True)
        self.correct_prediction = tf.equal(tf.argmax(self.logit_test, 1), tf.argmax(self.y, 1))
        self.predict = tf.argmax(self.logit_test, 1)
        self.accuracy = tf.reduce_mean(tf.cast(self.correct_prediction, tf.float32))

        """ Summary """
        self.loss_sum = tf.summary.scalar("loss", self.loss_cross_entropy)


    def train(self):

        # initialize all variables
        tf.global_variables_initializer().run()
#         sess.run(tf.global_variables_initializer())

        # saver to save model
        self.saver = tf.train.Saver(max_to_keep = self.max_to_keep)

        # summary writer
        self.writer = tf.summary.FileWriter(self.log_dir + '/' + self.model_name, self.sess.graph)

        # restore check-point if it exits
        could_load, checkpoint_epoch = self.load(self.checkpoint_dir)
        if could_load:
            start_epoch = (int)(checkpoint_epoch) + 1
            counter = 1
            f = open(self.trainhist_dir + '/' + self.model_name+'.pkl', 'rb') 
            self.train_hist = pickle.load(f)
            f.close()
            print(" [*] Load SUCCESS")
            print(" [!] START_EPOCH is ", start_epoch)
        else:
            start_epoch = 1
            counter = 1
            print(" [!] Load failed...")

        # loop for epoch
        start_time = time.time()
        for epoch_loop in range(start_epoch, self.epoch + 1):

            CNN_losses = []
  
            epoch_start_time = time.time()
            shuffle_idxs = random.sample(range(0, self.train_x.shape[0]), self.train_x.shape[0])
            shuffled_set = self.train_x[shuffle_idxs]
            shuffled_set_f1 = self.train_x_f1[shuffle_idxs]
            shuffled_set_f2 = self.train_x_f2[shuffle_idxs]
            shuffled_set_f3 = self.train_x_f3[shuffle_idxs]
            shuffled_label = self.train_y[shuffle_idxs]
    
            # get batch data
            for idx in range(self.num_batches_train):
                batch_x = shuffled_set[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f1 = shuffled_set_f1[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f2 = shuffled_set_f2[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_x_f3 = shuffled_set_f3[idx*self.batch_size:(idx+1)*self.batch_size]
                batch_y = shuffled_label[idx*self.batch_size:(idx+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                

                # update D network
                _, summary_str, cnn_loss = self.sess.run([self.cnn_optim, self.loss_sum, self.loss_cross_entropy],
                                               feed_dict={self.x: batch_x,
                                                          self.x_f1: batch_x_f1,
                                                          self.x_f2: batch_x_f2,
                                                          self.x_f3: batch_x_f3,
                                                          self.y: batch_y,
                                                          self.keep_prob: 0.5}
                                                      )
                self.writer.add_summary(summary_str, counter)

                CNN_losses.append(cnn_loss)

                # display training status
                counter += 1
                print("Epoch: [%2d] [%4d/%4d] time: %4.4f, loss: %.8f" % (epoch_loop, idx, self.num_batches_train, 
                                                                          time.time() - start_time, cnn_loss))

            # After an epoch
            # Evaluates accuracy on test set
            test_accuracy_list = []
            for idx_test in range(self.num_batches_test):
                batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
                batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                        [self.batch_size, self.nb_class])
                accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                    self.x_f1: batch_x_f1_test,
                                                                    self.x_f2: batch_x_f2_test,
                                                                    self.x_f3: batch_x_f3_test,
                                                                    self.y: batch_y_tes,
                                                                    self.keep_prob: 1.0})
                test_accuracy_list.append(accuracy)
            test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        
            #update learning rate
            _, rate = sess.run([self.add_global, self.lr])
            
            epoch_end_time = time.time()
            per_epoch_ptime = epoch_end_time - epoch_start_time
            
            print('[%d/%d] - ptime: %.4f loss: %.8f acc: %.5f lr: %.8f'% (epoch_loop, self.epoch, per_epoch_ptime, 
                                                                    np.mean(CNN_losses), test_accuracy, rate))
            
            self.train_hist['losses'].append(np.mean(CNN_losses))
            self.train_hist['accuracy'].append( test_accuracy)
            self.train_hist['learning_rate'].append(rate)
            self.train_hist['per_epoch_ptimes'].append(per_epoch_ptime)
            
            # save model
            self.save(self.checkpoint_dir, epoch_loop)
            
            # save trainhist for train
            f = open(self.trainhist_dir + '/' + self.model_name + '.pkl', 'wb') 
            pickle.dump(self.train_hist, f)
            f.close()
            self.show_train_hist(self.train_hist, save=True, path= self.trainhist_dir + '/' 
                                 + self.model_name + '.png')

        end_time = time.time()
        total_ptime = end_time - start_time
        self.train_hist['total_ptime'].append(total_ptime)
        print('Avg per epoch ptime: %.2f, total %d epochs ptime: %.2f' % (np.mean(self.train_hist['per_epoch_ptimes']), 
                                                                          self.epoch, total_ptime))
        print(" [*] Training finished!")
        
        """test after train"""
        best_acc = max(self.train_hist['accuracy'])
        beat_epoch = self.train_hist['accuracy'].index(best_acc) + 1
        print (" [*] Best Epoch: ", beat_epoch, ", Accuracy: ", best_acc)
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(beat_epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f3_test =self.test_x_f3[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.x_f3: batch_x_f3_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Best Epoch:", beat_epoch, ", accuracy: ",test_accuracy, "!")

    def test(self, epoch):
        path_model = self.checkpoint_dir + '/' + self.model_name + '/'+ self.model_name +'-'+ str(epoch)

        """ restore epoch """
        new_saver = tf.train.import_meta_graph(path_model + '.meta' )
        new_saver.restore(self.sess,path_model)

        # Evaluates accuracy on test set
        test_accuracy_list = []
        for idx_test in range(self.num_batches_test):
            batch_x_test = self.test_x[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f1_test =self.test_x_f1[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_x_f2_test =self.test_x_f2[idx_test*self.batch_size:(idx_test+1)*self.batch_size]
            batch_y_tes = self.test_y[idx_test*self.batch_size:(idx_test+1)*self.batch_size].reshape(
                                    [self.batch_size, self.nb_class])
            accuracy = self.sess.run([self.accuracy],feed_dict={self.x: batch_x_test, 
                                                                self.x_f1: batch_x_f1_test,
                                                                self.x_f2: batch_x_f2_test,
                                                                self.y: batch_y_tes,
                                                                self.keep_prob: 1.0})
            test_accuracy_list.append(accuracy)
        test_accuracy = np.sum(test_accuracy_list)/self.num_batches_test
        print(" [*] Finished testing Epoch:", epoch, ", accuracy: ",test_accuracy, "!")
        
    def show_all_variables(self):
        model_vars = tf.trainable_variables()
        tf.contrib.slim.model_analyzer.analyze_vars(model_vars, print_info=True) 

    def save(self, checkpoint_dir, step):
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)

        self.saver.save(self.sess,os.path.join(checkpoint_dir, self.model_name), global_step=step)

    def load(self, checkpoint_dir):
        import re
        print(" [*] Reading checkpoints...")
        checkpoint_dir = os.path.join(checkpoint_dir, self.model_name)

        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            self.saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            epoch = int(next(re.finditer("(\d+)(?!.*\d)",ckpt_name)).group(0))
            print(" [*] Success to read [{}], epoch [{}]".format(ckpt_name,epoch))
            return True, epoch
        else:
            print(" [*] Failed to find a checkpoint")
            return False, 0
        
    def show_train_hist(self, hist, show = False, save = False, path = 'Train_hist.png'):
        x = range(1, len(hist['losses'])+1)

        y1 = hist['losses']
        y2 = hist['accuracy']
        
        fig, ax1 = plt.subplots()
                            
        ax2 = ax1.twinx()  

        ax1.plot(x, y1, 'b')
        ax2.plot(x, y2, 'r')
                            
        ax1.set_xlabel('Epoch')
                            
        ax1.set_ylabel('CNN_loss')    
        ax2.set_ylabel('accuracy')

        plt.grid(True)
        plt.tight_layout()

        if save:
            plt.savefig(path, dpi = 400)

        if show:
            plt.show()
        else:
            plt.close()
   

In [ ]:
dataset = '4_Flowers_1s'
epoch = 80
batch_size = 100
checkpoint_dir = 'checkpoint'
log_dir = 'logs'
trainhist_dir = 'trainhist'

if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)

# --log_dir
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    
# --trainhist_dir
if not os.path.exists(trainhist_dir):
    os.makedirs(trainhist_dir)

# open session
with tf.Session(config=tf.ConfigProto(allow_soft_placement=True)) as sess:
    
    # declare instance for GAN
    CNN = CNN(sess, epoch=epoch, batch_size=batch_size, dataset_name=dataset, checkpoint_dir=checkpoint_dir, 
                log_dir=log_dir, trainhist_dir=trainhist_dir)

    # build graph
    CNN.build_model()

    # show network architecture
    CNN.show_all_variables()

    # launch the graph in a session
    CNN.train()
    
#     CNN.test(epoch)
        
sess.close()
        
# lrdecay
# Avg per epoch ptime: 28.37, total 80 epochs ptime: 1796.05
#  [*] Training finished!
#  [*] Best Epoch:  34 , Accuracy:  0.753333330154419
# INFO:tensorflow:Restoring parameters from checkpoint/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay/CNN_Canny&Pyramid_L_C5_D1_Kernel(3,3)_128_lrdecay-34
#  [*] Finished testing Best Epoch: 34 , accuracy:  0.753333330154419 !

CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
CNN:x (100, 128, 128, 3)
CNN:x_f1 (100, 128, 128, 1)
CNN:x_f2 (100, 128, 128, 2)
CNN:x_f3 (100, 128, 128, 2)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 32)
CNN: (100, 128, 128, 128)
CNN: (100, 64, 64, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 32, 32, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 16, 16, 64)
CNN: (100, 8, 8, 64)
CNN: (100, 8, 8, 128)
CNN: (100, 4, 4, 128)
CNN: (100, 2048)
CNN:out_logit (100, 4)
------------------------
---------
Variables: name (type shape) [

Epoch: [20] [  27/  60] time: 45.1528, loss: 0.00829858
Epoch: [20] [  28/  60] time: 45.5897, loss: 0.01601372
Epoch: [20] [  29/  60] time: 46.0283, loss: 0.00566602
Epoch: [20] [  30/  60] time: 46.4651, loss: 0.00268182
Epoch: [20] [  31/  60] time: 46.9003, loss: 0.01513672
Epoch: [20] [  32/  60] time: 47.3372, loss: 0.01005945
Epoch: [20] [  33/  60] time: 47.7720, loss: 0.03290445
Epoch: [20] [  34/  60] time: 48.2064, loss: 0.00543980
Epoch: [20] [  35/  60] time: 48.6459, loss: 0.00513525
Epoch: [20] [  36/  60] time: 49.0845, loss: 0.02731247
Epoch: [20] [  37/  60] time: 49.5228, loss: 0.00329338
Epoch: [20] [  38/  60] time: 49.9594, loss: 0.01522183
Epoch: [20] [  39/  60] time: 50.3973, loss: 0.00124339
Epoch: [20] [  40/  60] time: 50.8321, loss: 0.01291889
Epoch: [20] [  41/  60] time: 51.2706, loss: 0.01100307
Epoch: [20] [  42/  60] time: 51.7077, loss: 0.00602346
Epoch: [20] [  43/  60] time: 52.1451, loss: 0.01041762
Epoch: [20] [  44/  60] time: 52.5815, loss: 0.0

Epoch: [22] [  51/  60] time: 112.2494, loss: 0.00129473
Epoch: [22] [  52/  60] time: 112.6879, loss: 0.01207004
Epoch: [22] [  53/  60] time: 113.1261, loss: 0.00164694
Epoch: [22] [  54/  60] time: 113.5673, loss: 0.01037128
Epoch: [22] [  55/  60] time: 114.0081, loss: 0.04443729
Epoch: [22] [  56/  60] time: 114.4487, loss: 0.00171637
Epoch: [22] [  57/  60] time: 114.8896, loss: 0.00860306
Epoch: [22] [  58/  60] time: 115.3282, loss: 0.04678701
Epoch: [22] [  59/  60] time: 115.7659, loss: 0.00494727
[22/80] - ptime: 27.8139 loss: 0.01323227 acc: 0.71667 lr: 0.00081000
Epoch: [23] [   0/  60] time: 118.1048, loss: 0.00196450
Epoch: [23] [   1/  60] time: 118.5431, loss: 0.00207718
Epoch: [23] [   2/  60] time: 118.9808, loss: 0.00537818
Epoch: [23] [   3/  60] time: 119.4187, loss: 0.00641534
Epoch: [23] [   4/  60] time: 119.8561, loss: 0.00946127
Epoch: [23] [   5/  60] time: 120.3247, loss: 0.02212361
Epoch: [23] [   6/  60] time: 120.7639, loss: 0.00208399
Epoch: [23] [   7/

Epoch: [25] [  12/  60] time: 179.6915, loss: 0.00421992
Epoch: [25] [  13/  60] time: 180.1288, loss: 0.03486260
Epoch: [25] [  14/  60] time: 180.5681, loss: 0.01000942
Epoch: [25] [  15/  60] time: 181.0063, loss: 0.02517147
Epoch: [25] [  16/  60] time: 181.4426, loss: 0.02529684
Epoch: [25] [  17/  60] time: 181.8808, loss: 0.00556881
Epoch: [25] [  18/  60] time: 182.3190, loss: 0.00597380
Epoch: [25] [  19/  60] time: 182.7576, loss: 0.00208326
Epoch: [25] [  20/  60] time: 183.1964, loss: 0.01756799
Epoch: [25] [  21/  60] time: 183.6352, loss: 0.01389788
Epoch: [25] [  22/  60] time: 184.0737, loss: 0.01947175
Epoch: [25] [  23/  60] time: 184.5109, loss: 0.00752302
Epoch: [25] [  24/  60] time: 184.9514, loss: 0.01488444
Epoch: [25] [  25/  60] time: 185.3903, loss: 0.02285243
Epoch: [25] [  26/  60] time: 185.8277, loss: 0.00431143
Epoch: [25] [  27/  60] time: 186.2678, loss: 0.00120646
Epoch: [25] [  28/  60] time: 186.7105, loss: 0.00458601
Epoch: [25] [  29/  60] time: 1

Epoch: [27] [  34/  60] time: 245.7434, loss: 0.01347816
Epoch: [27] [  35/  60] time: 246.1840, loss: 0.00531861
Epoch: [27] [  36/  60] time: 246.6239, loss: 0.06402098
Epoch: [27] [  37/  60] time: 247.0632, loss: 0.07551503
Epoch: [27] [  38/  60] time: 247.5014, loss: 0.01477065
Epoch: [27] [  39/  60] time: 247.9387, loss: 0.01823910
Epoch: [27] [  40/  60] time: 248.3753, loss: 0.04650196
Epoch: [27] [  41/  60] time: 248.8144, loss: 0.00539137
Epoch: [27] [  42/  60] time: 249.2518, loss: 0.02650045
Epoch: [27] [  43/  60] time: 249.6927, loss: 0.01969184
Epoch: [27] [  44/  60] time: 250.1311, loss: 0.01092900
Epoch: [27] [  45/  60] time: 250.5692, loss: 0.03404409
Epoch: [27] [  46/  60] time: 251.0073, loss: 0.02125763
Epoch: [27] [  47/  60] time: 251.4473, loss: 0.00397631
Epoch: [27] [  48/  60] time: 251.8856, loss: 0.00087857
Epoch: [27] [  49/  60] time: 252.3270, loss: 0.02367695
Epoch: [27] [  50/  60] time: 252.7657, loss: 0.01276015
Epoch: [27] [  51/  60] time: 2

...

Epoch: [69] [  42/  60] time: 1471.5158, loss: 0.02575384
Epoch: [69] [  43/  60] time: 1471.9568, loss: 0.00776264
Epoch: [69] [  44/  60] time: 1472.3946, loss: 0.01569185
Epoch: [69] [  45/  60] time: 1472.8344, loss: 0.12987421
Epoch: [69] [  46/  60] time: 1473.2735, loss: 0.01154997
Epoch: [69] [  47/  60] time: 1473.7111, loss: 0.08090631
Epoch: [69] [  48/  60] time: 1474.1526, loss: 0.15245119
Epoch: [69] [  49/  60] time: 1474.5939, loss: 0.01248570
Epoch: [69] [  50/  60] time: 1475.0330, loss: 0.02971453
Epoch: [69] [  51/  60] time: 1475.4766, loss: 0.01753026
Epoch: [69] [  52/  60] time: 1475.9137, loss: 0.03567667
Epoch: [69] [  53/  60] time: 1476.3538, loss: 0.07420374
Epoch: [69] [  54/  60] time: 1476.7918, loss: 0.16849503
Epoch: [69] [  55/  60] time: 1477.2315, loss: 0.00421893
Epoch: [69] [  56/  60] time: 1477.6650, loss: 0.01003052
Epoch: [69] [  57/  60] time: 1478.1039, loss: 0.02084560
Epoch: [69] [  58/  60] time: 1478.5444, loss: 0.04121995
Epoch: [69] [ 

Epoch: [72] [   0/  60] time: 1538.0270, loss: 0.00031795
Epoch: [72] [   1/  60] time: 1538.4659, loss: 0.00070280
Epoch: [72] [   2/  60] time: 1538.9052, loss: 0.00010312
Epoch: [72] [   3/  60] time: 1539.3465, loss: 0.00040125
Epoch: [72] [   4/  60] time: 1539.7881, loss: 0.00010751
Epoch: [72] [   5/  60] time: 1540.2251, loss: 0.00129667
Epoch: [72] [   6/  60] time: 1540.6644, loss: 0.00290984
Epoch: [72] [   7/  60] time: 1541.1043, loss: 0.00319488
Epoch: [72] [   8/  60] time: 1541.5433, loss: 0.00007989
Epoch: [72] [   9/  60] time: 1541.9836, loss: 0.00031227
Epoch: [72] [  10/  60] time: 1542.4211, loss: 0.00026398
Epoch: [72] [  11/  60] time: 1542.8591, loss: 0.00025838
Epoch: [72] [  12/  60] time: 1543.2978, loss: 0.00016843
Epoch: [72] [  13/  60] time: 1543.7373, loss: 0.00002983
Epoch: [72] [  14/  60] time: 1544.1769, loss: 0.00007936
Epoch: [72] [  15/  60] time: 1544.6174, loss: 0.00030261
Epoch: [72] [  16/  60] time: 1545.0592, loss: 0.00036344
Epoch: [72] [ 

Epoch: [74] [  19/  60] time: 1603.3366, loss: 0.00000944
Epoch: [74] [  20/  60] time: 1603.7804, loss: 0.00005738
Epoch: [74] [  21/  60] time: 1604.2238, loss: 0.00006410
Epoch: [74] [  22/  60] time: 1604.6675, loss: 0.00008609
Epoch: [74] [  23/  60] time: 1605.1084, loss: 0.00031660
Epoch: [74] [  24/  60] time: 1605.5505, loss: 0.00001840
Epoch: [74] [  25/  60] time: 1605.9923, loss: 0.00011265
Epoch: [74] [  26/  60] time: 1606.4332, loss: 0.00012517
Epoch: [74] [  27/  60] time: 1606.8738, loss: 0.00008505
Epoch: [74] [  28/  60] time: 1607.3140, loss: 0.00024316
Epoch: [74] [  29/  60] time: 1607.7569, loss: 0.00000069
Epoch: [74] [  30/  60] time: 1608.1995, loss: 0.00021397
Epoch: [74] [  31/  60] time: 1608.6393, loss: 0.00001698
Epoch: [74] [  32/  60] time: 1609.0829, loss: 0.00018875
Epoch: [74] [  33/  60] time: 1609.5232, loss: 0.00021934
Epoch: [74] [  34/  60] time: 1609.9648, loss: 0.00015752
Epoch: [74] [  35/  60] time: 1610.4047, loss: 0.00009098
Epoch: [74] [ 

Epoch: [76] [  38/  60] time: 1673.4634, loss: 0.00026138
Epoch: [76] [  39/  60] time: 1673.9033, loss: 0.00020362
Epoch: [76] [  40/  60] time: 1674.3440, loss: 0.00012593
Epoch: [76] [  41/  60] time: 1674.7848, loss: 0.00000725
Epoch: [76] [  42/  60] time: 1675.2216, loss: 0.00004132
Epoch: [76] [  43/  60] time: 1675.6617, loss: 0.00001588
Epoch: [76] [  44/  60] time: 1676.1028, loss: 0.00193412
Epoch: [76] [  45/  60] time: 1676.5423, loss: 0.00128880
Epoch: [76] [  46/  60] time: 1676.9800, loss: 0.00094712
Epoch: [76] [  47/  60] time: 1677.4196, loss: 0.00008117
Epoch: [76] [  48/  60] time: 1677.8567, loss: 0.00028398
Epoch: [76] [  49/  60] time: 1678.2948, loss: 0.00004059
Epoch: [76] [  50/  60] time: 1678.7310, loss: 0.00007420
Epoch: [76] [  51/  60] time: 1679.1684, loss: 0.00004478
Epoch: [76] [  52/  60] time: 1679.6070, loss: 0.00001828
Epoch: [76] [  53/  60] time: 1680.0451, loss: 0.00050086
Epoch: [76] [  54/  60] time: 1680.4832, loss: 0.00021477
Epoch: [76] [ 

Epoch: [78] [  57/  60] time: 1738.0564, loss: 0.00055745
Epoch: [78] [  58/  60] time: 1738.4955, loss: 0.00007987
Epoch: [78] [  59/  60] time: 1738.9339, loss: 0.00021834
[78/80] - ptime: 27.7712 loss: 0.00186850 acc: 0.69333 lr: 0.00047830
Epoch: [79] [   0/  60] time: 1741.2333, loss: 0.00718483
Epoch: [79] [   1/  60] time: 1741.6739, loss: 0.00002792
Epoch: [79] [   2/  60] time: 1742.1137, loss: 0.00569074
Epoch: [79] [   3/  60] time: 1742.5527, loss: 0.00035553
Epoch: [79] [   4/  60] time: 1742.9892, loss: 0.00121577
Epoch: [79] [   5/  60] time: 1743.4286, loss: 0.00068474
Epoch: [79] [   6/  60] time: 1743.8681, loss: 0.00202234
Epoch: [79] [   7/  60] time: 1744.3059, loss: 0.00032234
Epoch: [79] [   8/  60] time: 1744.7464, loss: 0.00033621
Epoch: [79] [   9/  60] time: 1745.1849, loss: 0.00326931
Epoch: [79] [  10/  60] time: 1745.6221, loss: 0.00024440
Epoch: [79] [  11/  60] time: 1746.0626, loss: 0.00013209
Epoch: [79] [  12/  60] time: 1746.5012, loss: 0.00048478
Ep

In [ ]:
import pygame

file='/home/huiqy/Music/CloudMusic/All_Time_Low.mp3' #文件名是完整路径名
pygame.mixer.init() #初始化音频
track = pygame.mixer.music.load(file)#载入音乐文件
pygame.mixer.music.play()#开始播放
time.sleep(60)#播放10秒
pygame.mixer.music.stop()#停止播放